# Proyecto ilustrativo.

```
.
├── build.gradle
├── gradle
│   └── wrapper
│       ├── gradle-wrapper.jar
│       └── gradle-wrapper.properties
├── gradlew
├── gradlew.bat
├── settings.gradle
└── src
    ├── main
    │   └── java
    │       └── com
    │           └── example
    │               ├── Person.java
    │               └── PersonController.java
    └── test
        └── java
            └── com
                └── example
                └── PersonControllerTest.java
```

## El *script* ```build.gradle```. 

```groovy
plugins {
  id 'java'
  id 'jacoco'
}

repositories {
  mavenCentral()
}

dependencies {
  // Dependencias para JUnit 5
  testImplementation 'org.junit.jupiter:junit-jupiter-api:5.8.2'
  testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.8.2'

  // Dependencia para Mockito
  testImplementation 'org.mockito:mockito-junit-jupiter:4.2.0'

  // Dependencia para Spark, un framework para crear APIs web
  implementation 'com.sparkjava:spark-core:2.9.3'
}

test {
  // Usar la plataforma de JUnit 5
  useJUnitPlatform()
}

jacocoTestReport {
  // Configurar el reporte de cobertura de código
  reports {
    xml.enabled false
    csv.enabled false
    html.destination file("${buildDir}/jacocoHtml")
  }
}
```

## La clase ```Person```.

La clase ```Person``` represente el recurso que se va a exponer en la *API* 

```java
public class Person {

  private String name;
  private int age;

  public Person() {
  // Constructor vacío
  }

  public Person(String name, int age) {
    this.name = name;
  this.age = age;
  }

  // Getters y setters

}
```

## La clase ```PersonController```.

Crear una clase que implemente la lógica de la *API* usando *Spark*. Por ejemplo, se puede crear una clase ```PersonController``` que tenga dos endpoints: uno para crear una persona con el método ```POST``` y otro para obtener o eliminar una persona por su nombre con los métodos ```GET``` y ```DELETE```

```java
import com.google.gson.Gson;
import spark.Request;
import spark.Response;

import java.util.HashMap;
import java.util.Map;

import static spark.Spark.*;

public class PersonController {

  private static final String JSON_TYPE = "application/json";
  private static final Gson gson = new Gson();

  // Un mapa que simula una base de datos en memoria
  private static final Map<String, Person> persons = new HashMap<> ();

  public static void main(String[] args) {
    // Definir el puerto del servidor
    port(8080);

    // Definir el endpoint para crear una persona con el método POST
    post("/persons", JSON_TYPE, (request, response) -> {
    
      // Obtener el cuerpo de la petición como un objeto Person
      Person person = gson.fromJson(request.body(), Person.class);

      // Validar que el nombre y la edad no sean nulos o vacíos
      if (person.getName() == null || person.getName().isEmpty() ||
    person.getAge() == null || person.getAge() <= 0) {
    
        // Devolver un código de error 400 (Bad Request) y un mensaje
        response.status(400);
        return "Invalid name or age";
      }

      // Guardar la persona en el mapa usando el nombre como clave
      persons.put(person.getName(), person);

      // Devolver un código de éxito 201 (Created) y la persona creada en formato JSON
      response.status(201);
      response.type(JSON_TYPE);
      return gson.toJson(person);
    });

    // Definir el endpoint para obtener o eliminar una persona por su nombre con los métodos GET y DELETE
    path("/persons/:name", () -> {

      // Definir el método GET para obtener una persona por su nombre
      get("", JSON_TYPE, (request, response) -> {
        // Obtener el nombre de la persona como parámetro de la ruta
        String name = request.params(":name");

        // Buscar la persona en el mapa por su nombre
        Person person = persons.get(name);

        // Si la persona no existe, devolver un código de error 404 (Not Found) y un mensaje
        if (person == null) {
          response.status(404);
          return "Person not found";
        }

        // Si la persona existe, devolver un código de éxito 200 (OK) y la persona en formato JSON
        response.status(200);
        response.type(JSON_TYPE);
        return gson.toJson(person);
      });

      // Definir el método DELETE para eliminar una persona por su nombre
      delete("", JSON_TYPE, (request, response) -> {
        // Obtener el nombre de la persona como parámetro de la ruta
        String name = request.params(":name");

        // Eliminar la persona del mapa por su nombre y obtener el         valor eliminado
        Person person = persons.remove(name);

        // Si la persona no existía, devolver un código de error 404 (Not Found) y un mensaje
        if (person == null) {
          response.status(404);
          return "Person not found";
        }

        // Si la persona existía, devolver un código de éxito 204 (No Content) y ningún cuerpo
        response.status(204);
        return "";
      });
    });
  }

  // Un método auxiliar para obtener el cuerpo de una petición como un objeto Person
  private static Person getPersonFromBody(Request request) {
    return gson.fromJson(request.body(), Person.class);
  }
}
```



1. 
Crear una clase que implemente las pruebas unitarias usando JUnit y Mockito. Por ejemplo, se puede crear una clase PersonControllerTest que tenga tres métodos de prueba: uno para crear una persona válida con el método POST, otro para obtener una persona existente con el método GET y otro para eliminar una persona existente con el método DELETE:

import com.google.gson.Gson;
import org.junit.jupiter.api.AfterAll;
import org.junit.jupiter.api.BeforeAll;
import org.junit.jupiter.api.Test;
import spark.Spark;

import java.io.IOException;
import java.net.HttpURLConnection;
import java.net.URL;
import java.nio.charset.StandardCharsets;
import java.util.Scanner;

import static org.junit.jupiter.api.Assertions.*;

public class PersonControllerTest {

private static final Gson gson = new Gson();

@BeforeAll
public static void setUp() {
// Iniciar el servidor antes de las pruebas
PersonController.main(null);
Spark.awaitInitialization();
}

@AfterAll
public static void tearDown() {
// Detener el servidor después de las pruebas
Spark.stop();
}

@Test
public void testCreateValidPersonWithPost() throws IOException {
// Crear una persona válida con el nombre "Alice" y la edad 25
Person person = new Person("Alice", 25);

// Convertir la persona a formato JSON
String json = gson.toJson(person);

// Crear una conexión HTTP al endpoint /persons con el método POST
HttpURLConnection connection = (HttpURLConnection) new URL("http://localhost:8080/persons").openConnection();
connection.setRequestMethod("POST");
connection.setRequestProperty("Content-Type", "application/json");
connection.setDoOutput(true);

// Escribir el JSON de la persona en el cuerpo de la petición
connection.getOutputStream().write(json.getBytes(StandardCharsets.UTF_8));

// Obtener el código de respuesta y el tipo de contenido
int responseCode = connection.getResponseCode();
String contentType = connection.getContentType();

// Leer el cuerpo de la respuesta como un JSON
Scanner scanner = new Scanner(connection.getInputStream());
String responseBody = scanner.nextLine();
scanner.close();

// Convertir el JSON a un objeto Person
Person responsePerson = gson.fromJson(responseBody, Person.class);

// Verificar que el código de respuesta sea 201 (Created)
assertEquals(201, responseCode);

// Verificar que el tipo de contenido sea application/json
assertEquals("application/json", contentType);

// Verificar que la persona devuelta sea igual a la persona creada
assertEquals(person, responsePerson);
}

@Test
public void testGetExistingPersonWithName() throws IOException {
// Crear una persona válida con el nombre "Bob" y la edad 30
Person person = new Person("Bob", 30);

// Convertir la persona a formato JSON
String json = gson.toJson(person);

// Crear una conexión HTTP al endpoint /persons con el método POST para crear la persona
HttpURLConnection postConnection = (HttpURLConnection) new URL("http://localhost:8080/persons").openConnection();
postConnection.setRequestMethod("POST");
postConnection.setRequestProperty("Content-Type", "application/json");
postConnection.setDoOutput(true);

// Escribir el JSON de la persona en el cuerpo de la petición
postConnection.getOutputStream().write(json.getBytes(StandardCharsets.UTF_8));

// Obtener el código de respuesta y verificar que sea 201 (Created)
int postResponseCode = postConnection.getResponseCode();
assertEquals(201, postResponseCode);

// Cerrar la conexión POST
postConnection.disconnect();

// Crear una conexión HTTP al endpoint /persons/Bob con el método GET para obtener la persona por su nombre
HttpURLConnection getConnection = (HttpURLConnection) new URL("http://localhost:8080/persons/Bob").

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2023.</p>